In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import openpyxl
from datetime import datetime
import time # Añadido por si necesitamos pausas explícitas
import re

# Función para expandir un shadowRoot
def expand_shadow_element(driver, element):
    return driver.execute_script("return arguments[0].shadowRoot", element)

def extraer_y_guardar_valor_unidad_selenium():
    url = "https://www.bbva.com.co/personas/productos/inversion/fondos/pais.html"
    excel_filename = "ValorUnidadFondo.xlsx"

    # --- NO CAMBIES ESTAS RUTAS SI YA FUNCIONAN ---
    #driver_path = "C:/Users/Andrés/Desktop/Prueba analista 3/web scraping/chromedriver.exe" # Asegúrate que esta sea tu ruta CORRECTA y ABSOLUTA
    # O si lo tienes en la misma carpeta:
    # driver_path = "./chromedriver.exe" # Windows
    # driver_path = "./chromedriver"     # macOS/Linux

    #service = Service(executable_path=driver_path)

    options = webdriver.ChromeOptions()
    # Mantenemos --headless COMENTADO por ahora para que el navegador se vea y nos dé pistas visuales
    # options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--log-level=3")
    options.add_argument("--start-maximized") # Opcional: Inicia el navegador maximizado
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36") # Añadir un user-agent para simular un navegador real

    driver = webdriver.Chrome(options=options)
    print(f"Intentando acceder a la URL con Selenium: {url}")
    valor_unidad_float = None

    try:
        driver.get(url)
        time.sleep(15)

        print("Navegador iniciado y página cargando...")
        
        #Esperar y acceder al iframe
        iframebase = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "iframe__base"))
        )
        driver.switch_to.frame(iframebase)
        #buscar el contenedor padre del shadowroot para luego expandirlo
        # Esperar a que el contenedor del Shadow DOM esté presente
        fichaco = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "cells-template-fichaco"))
        )
        shadow_root = expand_shadow_element(driver, fichaco)
        
        liquidativo_rentabilidad = shadow_root.find_element(By.CLASS_NAME, "liquidativo-rentabilidad")
        print("Etiqueta de valor de unidad encontrada y visible en la página.")
        print(liquidativo_rentabilidad.get_attribute("outerHTML"))

        # El resto de la extracción y limpieza del texto es igual
        # Eliminar todo excepto números y comas/puntos
        print(f"Texto limpio: '{liquidativo_rentabilidad.get_attribute("innerHTML")}'")

        textolimpio = re.sub(r"[^\d,\.]", "", liquidativo_rentabilidad.get_attribute("innerHTML"))
        print(f"Texto limpio: '{textolimpio}'")
        # Convertir formato colombiano/europeo 22.599,18 a 22599.18
        numero = float(textolimpio.replace(".", "").replace(",", "."))

        print(numero)
        print(f"Texto bruto extraído por Selenium: '{numero}'")

        try:
            valor_unidad_float = float(numero)
            print(f"Valor de la unidad convertido a float: {valor_unidad_float:,.2f} Pesos")
        except ValueError:
            print(f"Error: No se pudo convertir '{numero}' a número.")
            valor_unidad_float = None
        
        # Esperar a que el accordion esté presente
        accordion = WebDriverWait(shadow_root, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "accordion"))
        )
        # Esperar a que existan al menos 4 accordion-items
        WebDriverWait(accordion, 10).until(
            lambda d: len(accordion.find_elements(By.CLASS_NAME, "accordion-item")) >= 4
        )

        # Seleccionar el cuarto item
        item = accordion.find_elements(By.CLASS_NAME, "accordion-item")[3]
        # Buscar el botón dentro del título
        titulo = item.find_element(By.CLASS_NAME, "accordion-titulo")

        # Esperar a que el botón bbva-button-action esté presente
        boton = WebDriverWait(titulo, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "bbva-button-action"))
        )

        # Paso 3: Hacer clic (opción segura)
        print("Clickeando boton documentacion")
        boton.click()
        
        accordionContenido = accordion.find_element(By.ID, "documentacion")
        iconodocumento = accordionContenido.find_element(By.CLASS_NAME, "icono-documento")
        fichaTecnica = iconodocumento.find_element(By.TAG_NAME, "bbva-button-action")
        print("Clickeando icono de ficha tecnica")
        fichaTecnica.click()
        print("Esperando ventana del pdf")

        # Guardar el handle actual
        main_window = driver.current_window_handle

        # Esperar a que se abra la nueva ventana/pestaña
        WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)

        # Cambiar al nuevo tab
        new_window = [w for w in driver.window_handles if w != main_window][0]
        driver.switch_to.window(new_window)

        # Tiempo de espera para que cargue la pagina
        time.sleep(2)
        pdf_url = driver.current_url
        
        print(f"URL del PDF: {pdf_url}")

        # Ruta pa guardar el archivo pdf
        ruta_destino = "ficha_tecnica.pdf"

        respuesta = requests.get(pdf_url)

        # Guardar el archivo si la respuesta es exitosa
        if respuesta.status_code == 200:
            with open(ruta_destino, "wb") as f:
                f.write(respuesta.content)
            print("PDF descargado correctamente.")
        else:
            print(f"Error al descargar el PDF. Código: {respuesta.status_code}")

        time.sleep(5)
    except Exception as e:
        print(f"Error durante la extracción con Selenium: {e}")
        valor_unidad_float = None

    finally:
        if driver:
            driver.quit()
            print("Navegador cerrado.")

    # 6. Guardar el dato en un archivo Excel (esta parte es igual al código anterior)
    try:
        workbook = openpyxl.Workbook()
        sheet = workbook.active
        sheet.title = "Valor Unidad"
        sheet['A1'] = "Fecha y Hora de Extracción"
        sheet['B1'] = "Valor de la Unidad (COP)"
        sheet['A2'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        if valor_unidad_float is not None:
            sheet['B2'] = valor_unidad_float
        else:
            sheet['B2'] = "No disponible"
        sheet.column_dimensions['A'].width = 25
        sheet.column_dimensions['B'].width = 25
        workbook.save(excel_filename)
        print(f"El valor se ha guardado correctamente en '{excel_filename}'")
    except Exception as e:
        print(f"Error al guardar el archivo Excel: {e}")

if __name__ == "__main__":
    extraer_y_guardar_valor_unidad_selenium()